In [1]:
import random
import pandas as pd
import numpy as np

In [2]:
listings = pd.read_csv('../../airbnb/listings.csv', encoding='utf-8');
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17074 entries, 0 to 17073
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            17074 non-null  int64  
 1   listing_url                                   17074 non-null  object 
 2   scrape_id                                     17074 non-null  int64  
 3   last_scraped                                  17074 non-null  object 
 4   name                                          17065 non-null  object 
 5   description                                   16169 non-null  object 
 6   neighborhood_overview                         9459 non-null   object 
 7   picture_url                                   17074 non-null  object 
 8   host_id                                       17074 non-null  int64  
 9   host_url                                      17074 non-null 

In [3]:
listings.drop(columns=["scrape_id", 
                        "last_scraped", 
                        "bathrooms", 
                        "calendar_updated", 
                        "availability_30", 
                        "availability_60", 
                        "availability_90", 
                        "availability_365", 
                        "has_availability", 
                        "calendar_last_scraped", 
                        "number_of_reviews", 
                        "number_of_reviews_ltm", 
                        "number_of_reviews_l30d", 
                        "first_review", 
                        "last_review", 
                        "license", 
                        "calculated_host_listings_count", 
                        "calculated_host_listings_count_entire_homes", 
                        "calculated_host_listings_count_private_rooms", 
                        "calculated_host_listings_count_shared_rooms", 
                        "reviews_per_month", 
                        "neighbourhood_group_cleansed"], inplace=True); # Drop columns that are not useful for our analysis

In [4]:
# Add host information to the listings
listings.loc[listings.host_listings_count.isna(), ['host_name']] = "Jakchai"
listings.loc[listings.host_listings_count.isna(), ['host_since']] = "2014-01-01"
listings.loc[listings.host_listings_count.isna(), ['host_location']] = "Bangkok, Thailand"
listings.loc[listings.host_listings_count.isna(), ['host_is_superhost']] = "f"
listings.loc[listings.host_listings_count.isna(), ['host_thumbnail_url']] = "https://a0.muscache.com/im/users/21348778/profile_pic/1410835018/original.jpg?aki_policy=profile_small"
listings.loc[listings.host_listings_count.isna(), ['host_picture_url']] = "https://a0.muscache.com/im/users/21348778/profile_pic/1410835018/original.jpg?aki_policy=profile_x_medium"
listings.loc[listings.host_listings_count.isna(), ['host_total_listings_count']] = 2
listings.loc[listings.host_listings_count.isna(), ['host_has_profile_pic']] = 't'
listings.loc[listings.host_listings_count.isna(), ['host_identity_verified']] = 't'
listings.loc[listings.host_listings_count.isna(), ['host_listings_count']] = 2

In [5]:
for column in ['minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm']:
    mean = listings[column].mean()
    listings[column].fillna(mean, inplace=True)

for column in ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value']:
    listings[column].fillna(0, inplace=True)

In [6]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17074 entries, 0 to 17073
Data columns (total 52 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           17074 non-null  int64  
 1   listing_url                  17074 non-null  object 
 2   name                         17065 non-null  object 
 3   description                  16169 non-null  object 
 4   neighborhood_overview        9459 non-null   object 
 5   picture_url                  17074 non-null  object 
 6   host_id                      17074 non-null  int64  
 7   host_url                     17074 non-null  object 
 8   host_name                    17074 non-null  object 
 9   host_since                   17074 non-null  object 
 10  host_location                17032 non-null  object 
 11  host_about                   9833 non-null   object 
 12  host_response_time           11445 non-null  object 
 13  host_response_ra

In [7]:
listings.to_csv('../../airbnb/listings_clean.csv', index=False, encoding='utf-8')

In [8]:
reviews = pd.read_csv('../../airbnb/reviews.csv', encoding='utf-8')

In [9]:
calendar = pd.read_csv('../../airbnb/calendar.csv', encoding='utf-8')

Generating Transactions:

In [10]:
# transactions = pd.DataFrame(columns=['id', 'listing_id', 'start_date', 'end_date', 'price', 'customer_id', 'comments'])
transactions = pd.read_csv('../../airbnb/reviews.csv', encoding='utf-8')
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246644 entries, 0 to 246643
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   listing_id     246644 non-null  int64 
 1   id             246644 non-null  int64 
 2   date           246644 non-null  object
 3   reviewer_id    246644 non-null  int64 
 4   reviewer_name  246642 non-null  object
 5   comments       246620 non-null  object
dtypes: int64(3), object(3)
memory usage: 11.3+ MB


In [11]:
transactions.head(5)

listing_id         id        date  reviewer_id reviewer_name  \
0    39544047  564106023  2019-11-15    273940743           Kim   
1    39544047  566622127  2019-11-20    143251478        Oliver   
2    39544047  568138380  2019-11-24    288635322         Doris   
3    39544047  568816394  2019-11-24    163790201          Yang   
4    39544047  597651173  2020-01-26    317920443         Kunat   

                                            comments  
0  Good communication, basic and clean accommodation  
1  Good, clean, cheap accomodation in proximity t...  
2  A nice room, very close to impact arena, also ...  
3  The landlord sister is really kind and lovely....  
4  The room was very new, cleand and well funishe...

In [12]:
transactions['end_date'] = pd.to_datetime(transactions['date']) - pd.to_timedelta(np.random.randint(1, 15, len(transactions)) , unit='days')
transactions['start_date'] = transactions['end_date'] -  pd.to_timedelta(np.random.randint(1, 8, len(transactions)) , unit='days')
transactions['total_days'] = (transactions['end_date'] - transactions['start_date']).dt.days

price = calendar.loc[calendar['listing_id'].isin(transactions['listing_id']), ['listing_id', 'adjusted_price']]
price = price.drop_duplicates(subset=['listing_id'],keep='first')
price['adjusted_price'] = price['adjusted_price'].str[1:].str.replace(',', '').astype(float)
transactions = transactions.merge(price, left_on='listing_id', right_on='listing_id', how='left')
transactions['total_price'] = transactions['adjusted_price'] * transactions['total_days']


transactions=transactions.drop(columns=['reviewer_name', 'date', 'total_days', 'adjusted_price'])
transactions

listing_id                  id  reviewer_id  \
0         39544047           564106023    273940743   
1         39544047           566622127    143251478   
2         39544047           568138380    288635322   
3         39544047           568816394    163790201   
4         39544047           597651173    317920443   
...            ...                 ...          ...   
246639    25650511  546028757839670443      9395774   
246640    25650511  575030942853820235    112523712   
246641    25650511  580846849788103547    301320803   
246642    25650511  619167599448170428    453549937   
246643    25650511  628621898000371278     21756179   

                                                 comments   end_date  \
0       Good communication, basic and clean accommodation 2019-11-01   
1       Good, clean, cheap accomodation in proximity t... 2019-11-17   
2       A nice room, very close to impact arena, also ... 2019-11-16   
3       The landlord sister is really kind and lovely.... 2019-11-10   
4       The room was very new, cleand and well funishe... 2020-01-22   
...                                                   ...        ...   
246639  Great place! Loved the gym and the pool. LOVED... 2022-01-12   
246640  綺麗に整理整頓されており食器類あり必要なものはすべて揃っています窓からの眺めもいいです おす... 2022-02-20   
246641  Frank was very kind and patient, I am so happy... 2022-02-27   
246642  The place exactly the same as photos, very nic... 2022-04-20   
246643  Great host. He gave me the flexibility of late... 2022-05-12   

       start_date  total_price  
0      2019-10-31        600.0  
1      2019-11-12       3000.0  
2      2019-11-13       1800.0  
3      2019-11-04       3600.0  
4      2020-01-21        600.0  
...           ...          ...  
246639 2022-01-09       2100.0  
246640 2022-02-18       1400.0  
246641 2022-02-20       4900.0  
246642 2022-04-18       1400.0  
246643 2022-05-10       1400.0  

[246644 rows x 7 columns]

In [13]:
transactions.describe()

listing_id            id   reviewer_id   total_price
count  2.466440e+05  2.466440e+05  2.466440e+05  2.466440e+05
mean   3.113233e+15  4.699634e+16  1.162459e+08  6.560128e+03
std    4.239635e+16  1.577644e+17  9.898522e+07  1.375864e+04
min    2.793400e+04  1.802610e+05  1.000000e+00  2.800000e+02
25%    9.849435e+06  2.087664e+08  3.361963e+07  2.277000e+03
50%    1.798931e+07  4.015605e+08  9.126269e+07  4.200000e+03
75%    2.915136e+07  5.634982e+08  1.782077e+08  7.400000e+03
max    6.480230e+17  6.539921e+17  4.646752e+08  1.120000e+06

In [14]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 246644 entries, 0 to 246643
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   listing_id   246644 non-null  int64         
 1   id           246644 non-null  int64         
 2   reviewer_id  246644 non-null  int64         
 3   comments     246620 non-null  object        
 4   end_date     246644 non-null  datetime64[ns]
 5   start_date   246644 non-null  datetime64[ns]
 6   total_price  246644 non-null  float64       
dtypes: datetime64[ns](2), float64(1), int64(3), object(1)
memory usage: 15.1+ MB


In [15]:
transactions.to_csv('../../airbnb/transactions.csv', index=False, encoding='utf-8')